In [1]:
import pandas as pd
df = pd.read_csv('daset.csv')

In [3]:
df

,label,text
0,0,@@CITATION will likely require a fundamental r...
1,0,Recent work on the integration of heterogeneou...
2,0,Recent advances in the application of machine ...
3,0,While the current generation of Dynamic Stocha...
4,0,@@CITATION provides a crucial theoretical fram...
5,0,@@CITATION will provide a comprehensive theore...
6,0,The burgeoning integration of algorithmic trad...
7,0,@@CITATION provides a compelling framework for...
8,0,Recent advances in heterogeneous agents New Ke...
9,0,@@CITATION have recently begun to explore the ...


In [ ]:
laliste=[]
for elt in df['text']:
    laliste.append(elt)

@@CITATION will likely require a fundamental re-evaluation of the Phillips curve framework, as the persistent decoupling of wage growth from unemployment rates in advanced economies suggests structural shifts in labor market dynamics that existing models fail to capture.
Recent work on the integration of heterogeneous agent models with deep learning techniques offers a promising avenue for more accurately simulating market dynamics and forecasting systemic risk ( @@CITATION ).
Recent advances in the application of machine learning to high-frequency financial data have begun to illuminate the complex, non-linear dynamics of market microstructure, yet a unified theoretical framework capable of integrating these empirical findings remains elusive; the development of such a framework, which must reconcile the statistical properties of algorithmic trading with foundational principles of market efficiency and price discovery, represents a critical direction for future research @@CITATION.
Wh

In [ ]:
# Requires transformers>=4.51.0
# Requires sentence-transformers>=2.7.0

from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-8B")

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-8B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

# The queries and documents to embed
queries = [
    "What is the capital of China?",
    "Explain gravity",
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7493, 0.0751],
#         [0.0880, 0.6318]])
